# M_background Sweep

Parameter sweep on `M_background`.

In [ ]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging

# utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([1]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)


In [ ]:
train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')

train_config = utils.getDictFromYamlFilename(train_config_file)




dataset_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', "caterpillar_baymax_starbot_all_front_single_only.yaml")
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

logging_dir = "code/data_volume/pdc/trained_models/M_background_sweep"num_iterations = 3500

num_image_pairs = 100
debug = False

TRAIN = True
EVALUATE = True

d = 3
network_dict = dict()

# num_image_pairs = 10
# num_iterations = 10

## Sweep on M_background

In [ ]:
dataset_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', "caterpillar_baymax_starbot_all_front_single_only.yaml")
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

logging_dir = "code/data_volume/pdc/trained_models/M_background_sweep"
M_background_list = [0.1, 0.5, 1.0, 1.5, 2.5]
for M_background in M_background_list:
    
    dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

    dataset = SpartanDataset(config=dataset_config)
    train_config = utils.getDictFromYamlFilename(train_config_file)


    name = "M_background_.%3f_%d" %(M_background, d)
    print "training %s" %(name)
    train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
    train._config["training"]["logging_dir"] = logging_dir
    train._config["training"]["logging_dir_name"] = name
    train._config["training"]["num_iterations"] = num_iterations
    train._config["dense_correspondence_network"]["descriptor_dimension"] = d
    
    train._config["loss_function"]["M_background"] = M_background


    if TRAIN:
        train.run()
    print "finished training descriptor of dimension %d" %(d)

     # now do evaluation
    print "running evaluation on network %s" %(name)
    model_folder = os.path.join(logging_dir, name)
    model_folder = utils.convert_to_absolute_path(model_folder)
    network_dict[name] = model_folder
    if EVALUATE:
        DCE = DenseCorrespondenceEvaluation
        DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
    print "finished running evaluation on network %s" %(name)
    